In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
# Hyperparameters
latent_dim = 100
batch_size = 64
learning_rate = 0.0002
epochs = 50

In [3]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to range [-1, 1]
])
mnist_data = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
data_loader = DataLoader(mnist_data, batch_size=batch_size, shuffle=True)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 57.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.91MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 15.0MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.44MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
# Generator Network
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z).view(-1, 1, 28, 28)


In [5]:
# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the 28x28 image to 784
        return self.model(x)


In [6]:
# Initialize models
generator = Generator(latent_dim)
discriminator = Discriminator()


In [7]:
# Loss function and optimizers
loss_fn = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(epochs):
    for real_images, _ in data_loader:
        batch_size = real_images.size(0)

        # Train Discriminator
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)

        real_images = real_images.to(torch.float32)
        real_loss = loss_fn(discriminator(real_images), real_labels)

        z = torch.randn(batch_size, latent_dim)
        fake_images = generator(z)
        fake_loss = loss_fn(discriminator(fake_images.detach()), fake_labels)

        d_loss = real_loss + fake_loss
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        z = torch.randn(batch_size, latent_dim)
        fake_images = generator(z)
        g_loss = loss_fn(discriminator(fake_images), real_labels)

        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

    # Print losses every epoch
    print(f"Epoch {epoch+1}/{epochs}, D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}")

    # Generate and save images every 10 epochs
    if (epoch + 1) % 10 == 0:
        z = torch.randn(16, latent_dim)
        samples = generator(z).detach()
        samples = (samples + 1) / 2  # Rescale to [0, 1]
        grid = torch.cat([torch.cat([samples[i * 4 + j] for j in range(4)], dim=2) for i in range(4)], dim=1)
        plt.imshow(grid.numpy().squeeze(), cmap="gray")
        plt.title(f"Generated Images at Epoch {epoch+1}")
        plt.show()

Epoch 1/50, D_loss: 0.1919, G_loss: 3.7536
Epoch 2/50, D_loss: 0.7403, G_loss: 2.2801
Epoch 3/50, D_loss: 0.1292, G_loss: 3.4361
Epoch 4/50, D_loss: 0.5410, G_loss: 5.9840
Epoch 5/50, D_loss: 0.0934, G_loss: 5.5936
Epoch 6/50, D_loss: 0.1315, G_loss: 6.4135
Epoch 7/50, D_loss: 0.0723, G_loss: 6.3991
